## GRV API

- acc email: maxharper26@icloud.com
- acc password: kowGot-sonbyk-4fazko
- api key: 4157732f-fa7f-4459-ab26-089e4deab17a
- documentation: https://topaz.grv.org.au/docs/#/codes/GetTrack

In [1]:
import pandas as pd
from topaz import TopazAPI
import time
import requests
from requests.exceptions import HTTPError
from json import JSONDecodeError
import numpy as np

In [2]:
key = '4157732f-fa7f-4459-ab26-089e4deab17a'
key_m = '6a8ffdc7-ee7a-4795-9b98-607e3e838342'
key_p ='34b33827-3ca1-4c2b-a4c5-40fde1d99a84'


In [3]:
topaz_api = TopazAPI(key)

In [4]:
authority_codes = [ 'NSW', 'QLD', 'SA', 'TAS', 'VIC', 'WA','NT']
all_races = []
missed_run=False

for code in authority_codes:
    try:
        races = topaz_api.get_races(from_date='2024-03-17', to_date='2024-03-18', owning_authority_code=code)
        all_races.append(races)
    except Exception as e:
        print(f"Error fetching races for {code}: {e}")

all_races_df = pd.concat(all_races, ignore_index=True)
race_ids = list(all_races_df['raceId'].unique())
race_ids

[1006994033,
 1006994034,
 1006994035,
 1006994036,
 1006994037,
 1006994038,
 1006994039,
 1006994040,
 1006994041,
 1006994042,
 1007029048,
 1007029049,
 1007029050,
 1007029051,
 1007029052,
 1007029053,
 1007029054,
 1007029055,
 1007029056,
 1007029057,
 1007029058,
 1007029059,
 1007029060,
 1007029061,
 1007029062,
 1007029063,
 1007029064,
 1007029065,
 1007029066,
 1007029067,
 1007029068,
 1007461416,
 1007461419,
 1007461422,
 1007461425,
 1007461428,
 1007461431,
 1007461434,
 1007461437,
 1007461442,
 1007461445,
 1007461417,
 1007461420,
 1007461423,
 1007461426,
 1007461429,
 1007461432,
 1007461435,
 1007461438,
 1007461440,
 1007461443,
 1007461446,
 1007461448,
 1007461418,
 1007461421,
 1007461424,
 1007461427,
 1007461430,
 1007461433,
 1007461436,
 1007461439,
 1007461441,
 1007461444,
 1007461447,
 1007504487,
 1007504488,
 1007504489,
 1007504490,
 1007504491,
 1007504492,
 1007504493,
 1007504494,
 1007504495,
 1007504496,
 1007062625,
 1007062626,
 1007062628,

In [5]:
len(race_ids)

272

In [6]:
# race_ids = pd.read_csv('api_files/missing.csv')['missed']
# missed_run=True
# race_ids

In [7]:
big = ''
start = time.time()
first=0
headers={'X-API-Key':key}
missed=[]



for i in range(first,len(race_ids)):
    rID = race_ids[i]
    url = f'https://topaz.grv.org.au/api/race/{rID}/result'
    try:
        race = requests.get(url.format(rID),headers=headers).json()
    except JSONDecodeError as e:
        race={'error':1,'message':'empty json'}
    
    if('error' in race and 'message' in race):
        print(rID,race['message'])
        missed.append(rID)
    else:
        splits = race['splitTimes']
        
        
        summary = pd.DataFrame(race['runs'])

        cols=['track','distance','raceId','weightInKg','meetingDate','dogName','resultTime','bestFinishTrackAndDistance', 'pir','incomingGrade', 'outgoingGrade', 'rating','resultMarginLengths', 'startPaceCode', 'jumpCode',
           'startPrice','boxNumber']
        summary=summary[cols]
        summary['Plc'] = summary['resultTime'].rank(method='min')

        if(splits!=[]):
            splits = pd.DataFrame(splits)
            splits = splits[['dogName','splitTimeMarker','time']]

            markers = max(splits['splitTimeMarker'])
            for marker in range(1,markers+1):
                split2 = splits[splits['splitTimeMarker']==marker]
                split2=split2.rename(columns={'time':'split_'+str(marker)})[['dogName','split_'+str(marker)]]
                summary = pd.merge(summary,split2,on='dogName',how='outer')


        if(type(big)==str):
            big = summary
        else:
            big = pd.concat([big,summary])

            



end = time.time()
taken = end-start
print('time for',len(race_ids),':',round(taken,3),'avg:',round(taken/len(race_ids),2))


1007461456 empty json
1007461457 empty json
1007461458 empty json
1007461459 empty json
1007504500 empty json
1007504501 empty json
1007504502 empty json
1007504503 empty json
1007504504 empty json
1007504505 empty json
1007504506 empty json
1007504507 empty json
1007504508 empty json
1007504509 empty json
1007504510 empty json
1007504511 empty json
1007522523 empty json
1007522524 empty json
1007522525 empty json
1007522526 empty json
1007522527 empty json
1007522528 empty json
1007522529 empty json
1007522530 empty json
1007522531 empty json
1007522532 empty json
1007522533 empty json
1007522534 empty json
1006994021 empty json
1006994022 empty json
1006994023 empty json
1006994024 empty json
1006994025 empty json
1006994026 empty json
1006994027 empty json
1006994028 empty json
1006994029 empty json
1006994030 empty json
1006994031 empty json
1006994032 empty json
1007461460 empty json
1007461461 empty json
1007461462 empty json
1007461463 empty json
1007461464 empty json
1007461465

In [8]:
big

,track,distance,raceId,weightInKg,meetingDate,dogName,resultTime,bestFinishTrackAndDistance,pir,incomingGrade,...,rating,resultMarginLengths,startPaceCode,jumpCode,startPrice,boxNumber,Plc,split_1,split_2,split_3
0,Grafton,350,1006994033,NaN,2024-03-17T00:00:00.000Z,ORARA DOUG,NaN,NaN,None,Maiden,...,0,None,None,None,1.0,NaN,NaN,NaN,NaN,NaN
1,Grafton,350,1006994033,34.5,2024-03-17T00:00:00.000Z,BROOM BROOM,21.41,21.41,768,Maiden,...,0,15.00L,None,None,23.0,1.0,8.0,4.28,15.82,NaN
2,Grafton,350,1006994033,29.0,2024-03-17T00:00:00.000Z,MY REGARDS,21.16,21.16,647,Maiden,...,0,11.00L,None,None,8.0,2.0,7.0,4.24,15.66,NaN
3,Grafton,350,1006994033,30.9,2024-03-17T00:00:00.000Z,SYMI SURTEE,20.70,20.57,434,Maiden,...,100,4.50L,None,None,3.2,3.0,3.0,4.24,15.29,NaN
4,Grafton,350,1006994033,31.3,2024-03-17T00:00:00.000Z,GILBERT'S HELMET,20.41,20.41,572,7,...,0,1.00L,None,None,4.8,4.0,1.0,4.29,15.02,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Darwin,312,1006994020,30.1,2024-03-17T00:00:00.000Z,BLACK IRON,18.75,18.20,None,5,...,100,1.75L,None,None,3.8,1.0,3.0,NaN,NaN,NaN
2,Darwin,312,1006994020,31.8,2024-03-17T00:00:00.000Z,MANAWATA,18.83,18.03,None,5,...,95,2.75L,None,None,4.1,2.0,4.0,NaN,NaN,NaN
3,Darwin,312,1006994020,31.8,2024-03-17T00:00:00.000Z,AARON KEEPING,18.68,18.29,None,5,...,88,0.75L,None,None,23.0,4.0,2.0,NaN,NaN,NaN
4,Darwin,312,1006994020,33.2,2024-03-17T00:00:00.000Z,MORE BEERS,18.65,18.10,0,5,...,78,0.75L,None,None,6.0,5.0,1.0,7.33,11.32,NaN


In [9]:
big1 = big.copy()
big1['Track']=big1['track'].str.lower()
big1['dogName']=big1['dogName'].str.lower()

to_rename={
    'Plc':'Place',
    'dogName':'DogName',
    'boxNumber':'Box',
    'resultTime':'RunTime',
    'weightInKg':'Weight',
    'startPrice':'StartPrice',
    'pir':'PIR',
    'distance':'Distance',
    'raceId':'RaceId',
    'incomingGrade':'RaceGrade',
    'meetingDate':'date',
    'split_1':'Split'
}

big1=big1.rename(columns=to_rename)
big1 = big1[['Place','DogName','Box','Weight','StartPrice','PIR','Split','RunTime','RaceId','Distance','RaceGrade','Track','date']]
big1['TrainerLName']='poo'
big1['TrainerFName']='poo'

big1['date']=big1['date'].astype(str).str.split('T').str[0]
big1['date']=pd.to_datetime(big1['date'])


import os
files = os.listdir('api_files')
files = [x for x in files if 'missing' not in x and '.DS' not in x]
m=0
print(files)
for file in files:
    num = file.split('_')[1].split('.')[0]
    num=int(num)
    if(num>m):
        m=num
print(m)

big1.to_csv('api_files/api_'+str(m+1)+'.csv',index=False)
big1

['api_8.csv', 'api_9.csv', 'api_17.csv', 'api_16.csv', 'api_14.csv', 'api_28.csv', 'api_29.csv', 'api_15.csv', 'api_39.csv', 'api_11.csv', 'api_10.csv', 'api_38.csv', 'api_12.csv', 'api_13.csv', 'api_36.csv', 'api_22.csv', 'api_23.csv', 'api_37.csv', 'api_21.csv', 'api_35.csv', 'api_34.csv', 'api_20.csv', 'api_18.csv', 'api_24.csv', 'api_30.csv', 'api_31.csv', 'api_25.csv', 'api_19.csv', 'api_33.csv', 'api_27.csv', 'api_26.csv', 'api_32.csv', 'api_7.csv', 'api_41.csv', 'api_40.csv', 'api_6.csv', 'api_4.csv', 'api_42.csv', 'api_43.csv', 'api_5.csv', 'api_1.csv', 'api_2.csv', 'api_3.csv']
43


,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,NaN,orara doug,NaN,NaN,1.0,None,NaN,NaN,1006994033,350,Maiden,grafton,2024-03-17,poo,poo
1,8.0,broom broom,1.0,34.5,23.0,768,4.28,21.41,1006994033,350,Maiden,grafton,2024-03-17,poo,poo
2,7.0,my regards,2.0,29.0,8.0,647,4.24,21.16,1006994033,350,Maiden,grafton,2024-03-17,poo,poo
3,3.0,symi surtee,3.0,30.9,3.2,434,4.24,20.70,1006994033,350,Maiden,grafton,2024-03-17,poo,poo
4,1.0,gilbert's helmet,4.0,31.3,4.8,572,4.29,20.41,1006994033,350,7,grafton,2024-03-17,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,3.0,black iron,1.0,30.1,3.8,None,NaN,18.75,1006994020,312,5,darwin,2024-03-17,poo,poo
2,4.0,manawata,2.0,31.8,4.1,None,NaN,18.83,1006994020,312,5,darwin,2024-03-17,poo,poo
3,2.0,aaron keeping,4.0,31.8,23.0,None,NaN,18.68,1006994020,312,5,darwin,2024-03-17,poo,poo
4,1.0,more beers,5.0,33.2,6.0,0,7.33,18.65,1006994020,312,5,darwin,2024-03-17,poo,poo


In [10]:
if(missed_run==False):
    missing  = pd.read_csv('api_files/missing.csv')[['missed']]
    missing2 = pd.DataFrame(missed,columns=['missed'])
    missing3 = pd.concat([missing,missing2])
    missing4 = missing3['missed'].unique()
    missing5 = pd.DataFrame(missing4,columns=['missed'])
    missing5.to_csv('api_files/missing.csv',index=False)
else:
    missing5 = pd.DataFrame(missed,columns=['missed'])
    missing5.to_csv('api_files/missing.csv',index=False)
    
missing5

,missed
0,990395989
1,990395990
2,990395991
3,990395992
4,990395993
...,...
386,1007062655
387,1007062656
388,1007062657
389,1006994012


In [11]:
files = os.listdir('api_files')
files = [x for x in files if 'missing' not in x and '.DS' not in x]
files.sort()
files
merged = pd.read_csv('api_files/'+files[0])
for file in files[1:]:
    merged=pd.concat([merged,pd.read_csv('api_files/'+file)])

merged=merged.sort_values(by='date')
merged['Track']=np.where(merged['Track'].str.contains('dport @'),'hobart',merged['Track'])
merged

,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,6.0,chick magnet,1.0,30.5,5.5,566.0,NaN,19.700,990412069,336,5,muswellbrook,2024-01-28,poo,poo
1331,1.0,monkey's style,7.0,34.5,4.9,111.0,5.1800,24.282,992122290,435,5,sale,2024-01-28,poo,poo
1330,3.0,rumour confirmed,6.0,31.9,38.7,743.0,5.4425,24.537,992122290,435,5,sale,2024-01-28,poo,poo
1329,8.0,crew,5.0,34.5,57.1,678.0,5.4075,25.242,992122290,435,5,sale,2024-01-28,poo,poo
1328,7.0,persuasion,4.0,27.1,20.2,887.0,5.4775,25.181,992122290,435,5,sale,2024-01-28,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,NaN,lynette ann,NaN,NaN,1.0,NaN,NaN,NaN,1008720934,305,5,mount gambier,2024-03-21,poo,poo
388,1.0,quantum queen,1.0,26.3,3.4,NaN,11.2700,23.990,1008720935,400,6,mount gambier,2024-03-21,poo,poo
389,5.0,yadillah chief,2.0,34.0,11.0,NaN,11.6700,24.580,1008720935,400,7,mount gambier,2024-03-21,poo,poo
365,7.0,russian dylan,5.0,28.7,3.9,6.0,6.1800,17.860,1009069647,288,6,ipswich,2024-03-21,poo,poo


In [12]:
a = pd.read_csv('../allRaces2.csv')
b = merged.copy()
c = pd.concat([a,b])
#c = c.drop(columns=['Unnamed: 0.1'])
c = c.drop_duplicates(subset=['DogName','RunTime','date'],keep='first')

c['Track']=np.where(c['Track'].str.contains('murray'),'murray bridge',c['Track'])
c['Track']=np.where(c['Track'].str.contains('(ris)'),'richmond straight',c['Track'])
c['Track']=np.where(c['Track'].str.contains('(mep)'),'the meadows',c['Track'])
c['Track']=np.where(c['Track'].str.contains('(sap)'),'sandown park',c['Track'])



c.to_csv('../allRaces1.csv',index=False)
c

/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_73790/609129207.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c['Track']=np.where(c['Track'].str.contains('(ris)'),'richmond straight',c['Track'])
/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_73790/609129207.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c['Track']=np.where(c['Track'].str.contains('(mep)'),'the meadows',c['Track'])
/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_73790/609129207.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c['Track']=np.where(c['Track'].str.contains('(sap)'),'sandown park',c['Track'])


,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,1.0,raine allen,1.0,27.4,2.4,111,6.68,28.66,335811282,500,Restricted Win,bendigo,2018-07-01,Grenfell,C
1,2.0,surf a lot,2.0,32.8,6.3,332,7.01,28.82,335811282,500,Restricted Win,bendigo,2018-07-01,Tyley,C
2,3.0,pingin' bee,6.0,25.5,9.3,443,7.24,28.93,335811282,500,Restricted Win,bendigo,2018-07-01,Dapiran,P
3,4.0,lucas the great,7.0,32.2,9.1,655,7.31,29.03,335811282,500,Restricted Win,bendigo,2018-07-01,Hamilton,E
4,5.0,quavo,4.0,28.9,3.4,766,7.38,29.05,335811282,500,Restricted Win,bendigo,2018-07-01,Dapiran,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,NaN,lynette ann,NaN,NaN,1.0,NaN,NaN,NaN,1008720934,305,5,mount gambier,2024-03-21,poo,poo
388,1.0,quantum queen,1.0,26.3,3.4,NaN,11.27,23.99,1008720935,400,6,mount gambier,2024-03-21,poo,poo
389,5.0,yadillah chief,2.0,34.0,11.0,NaN,11.67,24.58,1008720935,400,7,mount gambier,2024-03-21,poo,poo
365,7.0,russian dylan,5.0,28.7,3.9,6.0,6.18,17.86,1009069647,288,6,ipswich,2024-03-21,poo,poo


In [13]:
## still missing 
for rID in missing5['missed']:
    url = f'https://topaz.grv.org.au/api/race/{rID}/runs'
    try:
        race = requests.get(url.format(rID),headers=headers).json()[0]
        print(race['track'],race['meetingDate'],race['distance'])
    except JSONDecodeError as e:
        print(rID,'fucked')
        
# race

Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 498
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Murray Bridge (MBR) 2024-02-04T00:00:00.000Z 395
Murray Bridge (MBR) 2024-02-04T00:00:00.000Z 395
Murr

Rockhampton 2024-03-20T00:00:00.000Z 407
Rockhampton 2024-03-20T00:00:00.000Z 407
Rockhampton 2024-03-20T00:00:00.000Z 407
Rockhampton 2024-03-20T00:00:00.000Z 407
Rockhampton 2024-03-20T00:00:00.000Z 407
Albion Park 2024-03-20T00:00:00.000Z 395
Albion Park 2024-03-20T00:00:00.000Z 395
Albion Park 2024-03-20T00:00:00.000Z 395
Albion Park 2024-03-20T00:00:00.000Z 395
Albion Park 2024-03-20T00:00:00.000Z 395
Albion Park 2024-03-20T00:00:00.000Z 395
Albion Park 2024-03-20T00:00:00.000Z 395
Albion Park 2024-03-20T00:00:00.000Z 520
Albion Park 2024-03-20T00:00:00.000Z 395
Albion Park 2024-03-20T00:00:00.000Z 395
Albion Park 2024-03-20T00:00:00.000Z 395
Albion Park 2024-03-20T00:00:00.000Z 395
Meadows (MEP) 2024-03-20T00:00:00.000Z 600
Meadows (MEP) 2024-03-20T00:00:00.000Z 525
Meadows (MEP) 2024-03-20T00:00:00.000Z 525
Traralgon 2024-03-20T00:00:00.000Z 350
Traralgon 2024-03-20T00:00:00.000Z 395
Traralgon 2024-03-20T00:00:00.000Z 395
Traralgon 2024-03-20T00:00:00.000Z 395
Traralgon 2024-03-

In [14]:
big1

,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,NaN,orara doug,NaN,NaN,1.0,None,NaN,NaN,1006994033,350,Maiden,grafton,2024-03-17,poo,poo
1,8.0,broom broom,1.0,34.5,23.0,768,4.28,21.41,1006994033,350,Maiden,grafton,2024-03-17,poo,poo
2,7.0,my regards,2.0,29.0,8.0,647,4.24,21.16,1006994033,350,Maiden,grafton,2024-03-17,poo,poo
3,3.0,symi surtee,3.0,30.9,3.2,434,4.24,20.70,1006994033,350,Maiden,grafton,2024-03-17,poo,poo
4,1.0,gilbert's helmet,4.0,31.3,4.8,572,4.29,20.41,1006994033,350,7,grafton,2024-03-17,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,3.0,black iron,1.0,30.1,3.8,None,NaN,18.75,1006994020,312,5,darwin,2024-03-17,poo,poo
2,4.0,manawata,2.0,31.8,4.1,None,NaN,18.83,1006994020,312,5,darwin,2024-03-17,poo,poo
3,2.0,aaron keeping,4.0,31.8,23.0,None,NaN,18.68,1006994020,312,5,darwin,2024-03-17,poo,poo
4,1.0,more beers,5.0,33.2,6.0,0,7.33,18.65,1006994020,312,5,darwin,2024-03-17,poo,poo
